In [1]:
USER_FLAG = "--user"
!pip3 install {USER_FLAG} google-cloud-aiplatform --upgrade
!pip3 install {USER_FLAG} kfp==2.4.0 google-cloud-pipeline-components
!pip3 install {USER_FLAG} gcsfs
!pip install -U google-cloud-aiplatform "shapely>2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 40.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-jupyter-plugin 0.1.80 requires pydantic~=1.10.0, but you have pydantic 2.12.5 which is incompatible.
ydata-profiling 4.6.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.12.5 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of google-cloud-pipeline-com

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


In [3]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"


KFP SDK version: 2.4.0
google_cloud_pipeline_components version: 2.8.0


In [2]:
import os
PROJECT_ID = ""
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)


Project ID:  qwiklabs-gcp-01-073e94ad7a2d


In [3]:
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"


In [4]:
!wget https://storage.googleapis.com/partner-usecase-bucket/ucase009/Bike-Sharing-Dataset.zip


--2025-12-26 06:12:00--  https://storage.googleapis.com/partner-usecase-bucket/ucase009/Bike-Sharing-Dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.207, 173.194.206.207, 74.125.126.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/zip]
Saving to: ‘Bike-Sharing-Dataset.zip’

Bike-Sharing-Datase 100%[===================>] 273.43K  --.-KB/s    in 0.004s  

2025-12-26 06:12:01 (63.9 MB/s) - ‘Bike-Sharing-Dataset.zip’ saved [279992/279992]



In [5]:
!unzip -q Bike-Sharing-Dataset.zip -d data


In [6]:
! gsutil cp ./data/hour.csv $BUCKET_NAME


Copying file://./data/hour.csv [Content-Type=text/csv]...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      


In [7]:
! gsutil ls -al $BUCKET_NAME


   1156736  2025-12-26T06:12:13Z  gs://qwiklabs-gcp-01-073e94ad7a2d-bucket/hour.csv#1766729533679944  metageneration=1
TOTAL: 1 objects, 1156736 bytes (1.1 MiB)


In [9]:
import os
import pprint as pp
import sys

import pickle
import os
import argparse

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
import numpy as np
import pandas as pd

from google.cloud import storage
from google.cloud import aiplatform 


In [10]:
url = BUCKET_NAME+"/hour.csv"
data = pd.read_csv(url)
data.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [11]:
modelling_columns = ["season" 
                     , "yr" 
                     ,"mnth" 
                     ,"hr" 
                     ,"holiday" 
                     , "weekday" 
                     , "workingday" 
                     , "weathersit" 
                     , "temp" 
                     , "atemp" 
                     , "hum" 
                     , "windspeed" 
                     , "casual" 
                     , "registered" 
                     , "cnt"
                    ]
data = data[modelling_columns]


In [12]:
train_size = 0.8
test_size = 0.1
valid_size = 0.1

train_ds, valid_ds, test_ds = np.split(data.sample(frac=1, random_state=42), [int((train_size)*len(data)), int((1-test_size)*len(data))])


In [13]:
target = "cnt"

x_train = train_ds.drop(columns=target, axis=1)
y_train = train_ds[target]

x_valid = valid_ds.drop(columns=target, axis=1)
y_valid = valid_ds[target]

x_test = test_ds.drop(columns=target, axis=1)
y_test = test_ds[target]


In [14]:
model = RandomForestRegressor()
model.fit(x_train , y_train)


RandomForestRegressor()

In [15]:
y_pred = model.predict(x_valid)

#evaluate Model
adj_r2 = r2_score(y_true=y_valid, y_pred=y_pred)
mae = mean_absolute_error(y_true=y_valid, y_pred=y_pred)
mse = mean_squared_error(y_true=y_valid, y_pred=y_pred)
mape = mean_absolute_percentage_error(y_true=y_valid, y_pred=y_pred)
rmse = np.sqrt(mse)
print(f"Adjusted R2 : {adj_r2}")
print(f"Mean Absolute Error : {mae}")
print(f"Mean Absolute Percentage Error : {round(mape,4)*100}%")
print(f"Mean Squared Error : {mse}")
print(f"Root Mean Squared Error : {rmse}")


Adjusted R2 : 0.9998008796510542
Mean Absolute Error : 1.0201611047180668
Mean Absolute Percentage Error : 0.48%
Mean Squared Error : 7.030162025316453
Root Mean Squared Error : 2.6514452710392598


In [16]:
MODEL_PATH=BUCKET_NAME+"/models/"
model_path = "./" + "model.pkl"
with open(model_path, 'wb') as file:  
    pickle.dump(model, file) 
    
#copy model artifacts to GCS storage
!gsutil cp "model.pkl" $MODEL_PATH


Copying file://model.pkl [Content-Type=application/octet-stream]...
- [1 files][ 69.8 MiB/ 69.8 MiB]                                                
Operation completed over 1 objects/69.8 MiB.                                     


In [17]:
#Prediction containers list available at : https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
serving_container_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"

#define GCS location for model artifacts
artifact_uri = MODEL_PATH

#Upload Model to Vertex AI Model Registry using Python SDK
model = aiplatform.Model.upload(display_name= "MLOps0-model" ,
                                    artifact_uri=artifact_uri,
                                    serving_container_image_uri=serving_container_uri)


Creating Model
Create Model backing LRO: projects/268112420118/locations/us-central1/models/5878630177369489408/operations/113654232040603648
Model created. Resource name: projects/268112420118/locations/us-central1/models/5878630177369489408@1
To use this Model in another session:
model = aiplatform.Model('projects/268112420118/locations/us-central1/models/5878630177369489408@1')


In [22]:
#Create the model endpoint using Python SDK
endpoint = model.deploy(machine_type="n1-standard-4",
                        min_replica_count=1,
                        max_replica_count=1)


Creating Endpoint
Create Endpoint backing LRO: projects/268112420118/locations/us-central1/endpoints/4022434647358046208/operations/3413068724670824448
Endpoint created. Resource name: projects/268112420118/locations/us-central1/endpoints/4022434647358046208
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/268112420118/locations/us-central1/endpoints/4022434647358046208')
Deploying model to Endpoint : projects/268112420118/locations/us-central1/endpoints/4022434647358046208
Deploy Endpoint model backing LRO: projects/268112420118/locations/us-central1/endpoints/4022434647358046208/operations/9149528750033993728
Endpoint model deployed. Resource name: projects/268112420118/locations/us-central1/endpoints/4022434647358046208


In [25]:
#Test the model endpoint using Python SDK

#create list to hold request data
instances = [
    [1.0, 0.0, 1.0, 0.0, 0.0, 6.0, 0.0, 1.0, 0.24, 0.2879, 0.81, 0.0, 3.0, 13.0],
  ]

prediction = endpoint.predict(instances=instances)

print(prediction)


Prediction(predictions=[16.05], deployed_model_id='8796443766417260544', metadata=None, model_version_id='1', model_resource_name='projects/268112420118/locations/us-central1/models/5878630177369489408', explanations=None)


In [28]:
# Undeploy the model and delete the endpoint
endpoint.undeploy_all()
endpoint.delete()
model.delete()


NotFound: 404 Endpoint projects/268112420118/locations/us-central1/endpoints/4022434647358046208 is not found.

In [29]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component , ClassificationMetrics , Metrics)

from kfp import compiler
from kfp.components import load_component_from_file

import json
import yaml



In [30]:
!mkdir components


In [32]:
@component(
    packages_to_install=["pandas", "pyarrow" , "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/download_data.yaml"
)
def download_data(input_data_path : str
                  , input_data_filename : str
                  , downloaded_data : Output[Dataset]):
    
    import pandas as pd
    import os
    
    print(f"input_data_path : {input_data_path}")
    print(f"input_data_filename : {input_data_filename}")
    print(f"downloaded_data : {downloaded_data}")
    print(f"downloaded_data.path : {downloaded_data.path}")
                
    url = os.path.join(input_data_path , input_data_filename)
    
    #read data from GCS location
    data = pd.read_csv(url)
    
    #write to output dataset path
    output_data_uri = downloaded_data.path + ".csv" 
    data.to_csv(output_data_uri 
                , index=False
                , encoding='utf-8-sig')

/var/tmp/ipykernel_3704/1800625246.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/1800625246.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def download_data(input_data_path : str


In [33]:
@component(
    packages_to_install=["pandas", "pyarrow", "fsspec" , "gcsfs"],
    base_image="python:3.9",
    output_component_file="./components/preprocess_data.yaml"
)
def preprocess_data(train_size : float 
                    , test_size : float
                    , valid_size : float
                    , train_data : Output[Dataset]
                    , valid_data : Output[Dataset]
                    , test_data : Output[Dataset]
                    , input_data:  Input[Dataset]
                   ):
    
    import numpy as np
    import pandas as pd
    
    print(f"train_size : {train_size}")
    print(f"test_size : {test_size}")
    print(f"valid_size : {valid_size}")
    print(f"train_data : {train_data}")
    print(f"valid_data : {valid_data}")
    print(f"test_data : {test_data}")
    print(f"input_data : {input_data}")
    
    data = pd.read_csv(input_data.path + ".csv")
    
    modelling_columns = ["season" 
                     , "yr" 
                     ,"mnth" 
                     ,"hr" 
                     ,"holiday" 
                     , "weekday" 
                     , "workingday" 
                     , "weathersit" 
                     , "temp" 
                     , "atemp" 
                     , "hum" 
                     , "windspeed" 
                     , "casual" 
                     , "registered" 
                     , "cnt"
                    ]
    
    data = data[modelling_columns]

    train_ds, valid_ds, test_ds = np.split(data.sample(frac=1, random_state=42), [int((train_size)*len(data)), int((1-test_size)*len(data))])
    train_ds.to_csv(train_data.path + ".csv" , index=False, encoding='utf-8-sig')
    valid_ds.to_csv(valid_data.path + ".csv" , index=False, encoding='utf-8-sig')
    test_ds.to_csv(test_data.path + ".csv" , index=False, encoding='utf-8-sig')


/var/tmp/ipykernel_3704/715708913.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/715708913.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def preprocess_data(train_size : float


In [34]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "click==8.1.7", "docstring-parser==0.16", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20"]
    , base_image="python:3.9"
    , output_component_file="./components/train.yaml"
)
def train_model(
    train_data:  Input[Dataset],
    model: Output[Model], 
):
    
    print(f"train_data : {train_data}")
    print(f"model : {model}")
    
    from sklearn.ensemble import RandomForestRegressor
    import pandas as pd
    import pickle
    import sklearn

    train_ds = pd.read_csv(train_data.path+".csv")
    my_model = RandomForestRegressor()
    
    target = "cnt"
    
    x_train = train_ds.drop(columns=target, axis=1)
    y_train = train_ds[target]
    
    my_model.fit(x_train , y_train)
    model.metadata["model_name"] = "RandomForestRegressor"
    model.metadata["framework"] = "sklearn"
    model.metadata["framework_version"] = sklearn.__version__
    file_name = model.path + f".pkl"
    
    with open(file_name, 'wb') as file:  
        pickle.dump(my_model, file)


/var/tmp/ipykernel_3704/1210268811.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/1210268811.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def train_model(


In [35]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs"]
    , base_image="python:3.9"
    , output_component_file="./components/evaluate_model.yaml"
)
def evaluate_model(
    test_data:  Input[Dataset],
    model: Input[Model], 
    target_column_name : str ,
    deployment_metric : str ,
    deployment_metric_threshold : float ,
    kpi: Output[Metrics]
)-> NamedTuple(
    "Outputs",
    [
        ("deploy_flag", str),  # Return parameter.
    ],
):
    
    print(f"test_data : {test_data}")
    print(f"model : {model}")
    print(f"kpi : {kpi}")
    print(f"deployment_metric : {deployment_metric}")
    print(f"deployment_metric_threshold : {deployment_metric_threshold}")
    
    from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
    import pandas as pd
    import pickle
    import numpy as np
    import json
    
    test_ds = pd.read_csv(test_data.path+".csv")
    target = target_column_name
    
    x_test = test_ds.drop(columns=target, axis=1)
    y_test = test_ds[target]
    
    print(f"model.path : {model.path}")
    file_name = model.path + f".pkl"
    print(f"file_name : {file_name}")
    #model = pickle.loads(file_name)
    with open(file_name, 'rb') as file:  
        model = pickle.load(file)
    
    y_pred = model.predict(x_test)
    r2 = r2_score(y_true=y_test, y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    mape = mean_absolute_percentage_error(y_true=y_test, y_pred=y_pred)
    rmse = np.sqrt(mse)
    
    model_metrics = {"r2" : r2 
                     , "mae" : mae 
                     , "mape" : mape 
                     , "mse" : mse 
                     , "rmse" : rmse
                    }
    
    print(f"Adjusted_R2 : {r2}")
    print(f"Mean Absolute Error : {mae}")
    print(f"Mean Absolute Percentage Error : {round(mape,4)*100}%")
    print(f"Mean Squared Error : {mse}")
    print(f"Root Mean Squared Error : {rmse}")
    
    kpi.log_metric("Adjusted_R2", float(r2))
    kpi.log_metric("Mean Absolute Error", float(mae))
    kpi.log_metric("Mean Absolute Percentage Error", float(mape))
    kpi.log_metric("Mean Squared Error", float(mse))
    kpi.log_metric("Root Mean Squared Error", float(rmse))
    
    actual_metric_value = model_metrics.get(deployment_metric)
    
    if actual_metric_value >= deployment_metric_threshold:
        deploy_flag = "True"
    else:
        deploy_flag = "False"
        
    return (deploy_flag,)


/var/tmp/ipykernel_3704/210292492.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/210292492.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def evaluate_model(


In [36]:
@component(
    packages_to_install=["pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs" , "google-cloud-aiplatform"]
    , base_image="python:3.9"
    , output_component_file="./components/register_model.yaml"
)
def register_model(
    serving_container_uri : str ,
    project_id : str ,
    region: str,
    model_name : str , 
    model: Input[Model], 
)-> NamedTuple(
    "Outputs",
    [
        ("model_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"serving_container_uri : {serving_container_uri}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    print(f"model : {model}")
    
    from google.cloud import aiplatform
    
    print(f"model.uri : {model.uri[:-5]}")
    
    aiplatform.init(project = project_id , location=region)
    model = aiplatform.Model.upload(display_name= model_name ,
                                    artifact_uri=model.uri[:-5],
                                    serving_container_image_uri=serving_container_uri)
    return (model.resource_name,)

/var/tmp/ipykernel_3704/142823085.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/142823085.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def register_model(


In [37]:
@component(
    packages_to_install=["kfp==2.4.0", "pandas", "pyarrow",  "scikit-learn==1.3.2" , "fsspec" , "gcsfs", "google-cloud-aiplatform", "click==8.1.7", "kfp-pipeline-spec==0.2.2", "kfp-server-api==2.0.5", "kubernetes==26.1.0", "PyYAML==6.0.2", "requests-toolbelt==0.10.1", "tabulate==0.9.0", "protobuf==3.20.3", "urllib3==1.26.20", "numpy==1.24.4", "google-cloud-pipeline-components==2.8.0"]
    , base_image="python:3.9"
    , output_component_file="./components/deploy_model.yaml"
)
def deploy_model(
    model_resource_name : str ,
    project_id : str ,
    region: str
)-> NamedTuple(
    "Outputs",
    [
        ("endpoint_resource_name", str),  # Return parameter.
    ],
):
    
    print(f"model_resource_name : {model_resource_name}")
    print(f"project_id : {project_id}")
    print(f"region : {region}")
    
    from google.cloud import aiplatform
    
    aiplatform.init(project = project_id , location=region)
    
    model = aiplatform.Model(model_resource_name)
    endpoint = model.deploy(machine_type="n1-standard-4",
                        min_replica_count=1,
                        max_replica_count=1)
    
    return (endpoint.resource_name,)


/var/tmp/ipykernel_3704/2844370707.py:1: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  @component(
/var/tmp/ipykernel_3704/2844370707.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  def deploy_model(


In [38]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)
print("Project ID: ", PROJECT_ID)
print("staging_bucket_uri: ",BUCKET_NAME)
print("input_data_path: ",BUCKET_NAME)


Service Account: 268112420118-compute@developer.gserviceaccount.com
Project ID:  qwiklabs-gcp-01-073e94ad7a2d
staging_bucket_uri:  gs://qwiklabs-gcp-01-073e94ad7a2d-bucket
input_data_path:  gs://qwiklabs-gcp-01-073e94ad7a2d-bucket


In [39]:
%%writefile config.json
{
    "project":"ENTER PROJECT_ID HERE",
    "region":"us-central1",
    "service_account":"ENTER SERVICE_ACCOUNT HERE",
    "staging_bucket_uri":"ENTER BUCKET_NAME HERE",
    "pipeline_name":"tabular-data-regression-kfp-cicd-pipeline",
    "pipeline_package_path":"tabular-data-regression-kfp-cicd-pipeline.json",
    "input_data_path":"ENTER BUCKET_NAME HERE",
    "input_data_filename":"hour.csv",
    "target_column_name":"cnt",
    "train_size":0.8,
    "test_size":0.1,
    "valid_size":0.1,
    "deployment_metric":"r2",
    "deployment_metric_threshold":0.8,
    "model_name":"model_tabular_regression",
    "serving_container_uri":"us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
}

Writing config.json


In [40]:
%%writefile build_pipeline.py

import json
import yaml

import kfp
from kfp import dsl
from kfp import compiler
from kfp.components import load_component_from_file

download_data = load_component_from_file("./components/download_data.yaml")
preprocess_data = load_component_from_file("./components/preprocess_data.yaml")
train_model = load_component_from_file("./components/train.yaml")
evaluate_model = load_component_from_file("./components/evaluate_model.yaml")
register_model = load_component_from_file("./components/register_model.yaml")
deploy_model = load_component_from_file("./components/deploy_model.yaml")

#read configuration from file
with open("config.json") as json_file:
    config = json.load(json_file)
    
PIPELINE_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to define the pipeline Context.
    name=PIPELINE_NAME,
    
)
def pipeline(project: str = "",
 region: str = "",
 service_account: str = "",
 staging_bucket_uri: str = "",
 pipeline_name: str = "",
 pipeline_package_path: str = "",
 input_data_path: str = "",
 input_data_filename: str = "",
 target_column_name: str = "",
 train_size: float = 0.8,
 test_size: float = 0.1,
 valid_size: float = 0.1,
 hypertune_container_image_uri: str = "" ,
 hypertune_machine_type: str = "",
 hypertune_machine_replica_count: int = 1 ,
 hypertune_max_trial_count: int = 1 ,
 hypertune_parallel_trial_count: int = 1 ,
 hypertune_metric : str = "" ,
 hypertune_metric_objective : str = "" ,
 hypertune_job_name: str = "" ,
 deployment_metric: str = "" ,
 deployment_metric_threshold: float = 0.8 ,
 serving_container_uri : str = "" ,
 model_name : str = "", 
 user_email : str = "",
 monitoring_job_name : str = "" ,
 predict_instance_schema_uri : str = ""
):
    
    download_data_op = download_data(input_data_path = input_data_path 
                                     , input_data_filename = input_data_filename
                                    )
    
    
    preprocess_data_op = preprocess_data(train_size = train_size
                                         , test_size = test_size
                                         , valid_size = valid_size
                                         , input_data = download_data_op.outputs["downloaded_data"])
    
    train_model_op = train_model(train_data = preprocess_data_op.outputs["train_data"])
    
    
    evaluate_model_op = evaluate_model(test_data = preprocess_data_op.outputs["test_data"]
                                       ,model = train_model_op.outputs["model"]
                                       ,target_column_name = target_column_name
                                       ,deployment_metric = deployment_metric 
                                       ,deployment_metric_threshold = deployment_metric_threshold
                                      )
    
    with dsl.If(evaluate_model_op.outputs["deploy_flag"] == "True"):
        
        register_model_op = register_model(serving_container_uri = serving_container_uri 
                                       , model = train_model_op.outputs["model"]
                                       , model_name = model_name
                                       , project_id = project 
                                       , region = region)
        
        #deploy only if metric value exceeds deployment threshold
        deploy_model_op = deploy_model(model_resource_name = register_model_op.outputs["model_resource_name"] 
                                   , project_id = project 
                                   , region = region)
    
compiler.Compiler().compile(
    pipeline_func=pipeline
    , package_path=PACKAGE_PATH
)


Writing build_pipeline.py


In [41]:
%%writefile run_pipeline.py

from google.cloud import aiplatform
import yaml
import json

with open("config.json") as json_file:
    config = json.load(json_file)
    
SERVICE_ACCOUNT = config.get("service_account")
DISPLAY_NAME = config.get("pipeline_name")
PACKAGE_PATH = config.get("pipeline_package_path")
BUCKET_URI = config.get("staging_bucket_uri")
PIPELINE_ROOT = "{}/pipeline_root/kfp_tabular_data_regression".format(BUCKET_URI)
print(f"PIPELINE_ROOT :{PIPELINE_ROOT}")

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=config,
)

job.submit(service_account = SERVICE_ACCOUNT)


Writing run_pipeline.py


In [42]:
!python3 build_pipeline.py

PIPELINE_ROOT :ENTER BUCKET_NAME HERE/pipeline_root/kfp_tabular_data_regression


In [43]:
!python3 run_pipeline.py

/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils
PIPELINE_ROOT :ENTER BUCKET_NAME HERE/pipeline_root/kfp_tabular_data_regression
Error when trying to get or create a GCS bucket for the pipeline output artifacts
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aiplatform/pipeline_jobs.py", line 464, in submit
    gcs_utils.create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist(
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aiplatform/utils/gcs_utils.py", line 302, in create_gcs_bucket_for_pipeline_artifacts_if_it_does_not_exist
    pipelines_bucket = bucket_from_uri(
  File "/home/jupyter/.local/lib/python3.10/site-packages/google/cloud/aipla

In [44]:
! gsutil cp ./config.json $BUCKET_NAME
! gsutil cp ./tabular-data-regression-kfp-cicd-pipeline.json $BUCKET_NAME
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/config.json
! gsutil acl ch -u AllUsers:R $BUCKET_NAME/tabular-data-regression-kfp-cicd-pipeline.json

Copying file://./config.json [Content-Type=application/json]...
/ [1 files][  706.0 B/  706.0 B]                                                
Operation completed over 1 objects/706.0 B.                                      
Copying file://./tabular-data-regression-kfp-cicd-pipeline.json [Content-Type=application/json]...
/ [1 files][ 31.5 KiB/ 31.5 KiB]                                                
Operation completed over 1 objects/31.5 KiB.                                     
Updated ACL on gs://qwiklabs-gcp-01-073e94ad7a2d-bucket/config.json
Updated ACL on gs://qwiklabs-gcp-01-073e94ad7a2d-bucket/tabular-data-regression-kfp-cicd-pipeline.json
